<a href="https://colab.research.google.com/github/UncleSamTech/301Lab1WalkThrough/blob/main/Copy_of_nltk_150.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize
import nltk
import time
import matplotlib.pyplot as plt
#import pandas as pd
import random
import scipy.stats as stats
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score


In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_eng Averaged Perceptron Tagger (JSON)
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] averaged_perceptron_tagger_rus Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ 

    Error loading P: Package 'P' not found in index



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> l

Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_eng Averaged Perceptron Tagger (JSON)
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] averaged_perceptron_tagger_rus Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... 

       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:

Collections:

([*] marks installed packages)

Download which package (l=list; x=cancel)?
  Identifier> x

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
scratch_model = None
ngram_model = None
loaded_scratch_model = None


In [ ]:
def train_mle(train_data,n,model_name):

        with open(train_data,"r",encoding="utf-8") as f:
            lines = f.readlines()
            random.shuffle(lines)
            lines = [line.replace("_", "UNDERSCORE").replace(">", "RIGHTANG").replace("<", "LEFTANG") for line in lines]

            tokenized_scratch_data = [list(word_tokenize(sent.strip())) for sent in lines]
            train_data_val,padded_sents = padded_everygram_pipeline(n,tokenized_scratch_data)

        try:
            scratch_model = MLE(n)
            scratch_model.fit(train_data_val,padded_sents)

            with open(f'{model_name}_{n}.pkl',"wb") as fd:
                pickle.dump(scratch_model,fd)
        except Exception as es:
            print("error as a result of ", es)

In [ ]:
def load_trained_model(model_name) :
        with open(model_name,"rb") as f:
            loaded_scratch_model = pickle.load(f)
            #print(type(self.loaded_scratch_model))
            #print(self.loaded_scratch_model.vocab)
            #print(self.loaded_scratch_model.counts("event_whenflagclicked"))
            #print(self.loaded_scratch_model.score("event_whenflagclicked"))
            #print(self.loaded_scratch_model.vocab.lookup("move"))
        return loaded_scratch_model

In [ ]:
def predict_next_scratch_token(model_name,context_data):
        loaded_model = load_trained_model(model_name)
        scratch_next_probaility_tokens = {}

        for prospect_token in loaded_model.vocab:
            #print(f"context data {context_data}")
            scratch_next_probaility_tokens[prospect_token] = loaded_model.score(prospect_token,context_data.split(" "))

        scratch_predicted_next_token = max(scratch_next_probaility_tokens,key=scratch_next_probaility_tokens.get)
        scratch_predicted_next_token = scratch_predicted_next_token.lower() if isinstance(scratch_predicted_next_token,str) else scratch_predicted_next_token
        #print("predicted score ", scratch_next_probaility_tokens)
        return scratch_predicted_next_token

In [ ]:
def scratch_evaluate_model_nltk(test_data,model_name):

        y_true = []
        y_pred = []

        with open(test_data,"r",encoding="utf-8") as f:
            lines= f.readlines()
            random.shuffle(lines)
            lines = [line.replace("_", "UNDERSCORE").replace(">", "RIGHTANG").replace("<", "LEFTANG") for line in lines]

            for line in lines:
                line = line.strip()
                sentence_tokens = line.split()

                context = ' '.join(sentence_tokens[:-1])  # Use all words except the last one as context
                true_next_word = sentence_tokens[-1].lower()

                predicted_next_word = predict_next_scratch_token(model_name,context)

                y_true.append(true_next_word)
                y_pred.append(predicted_next_word)


        #self.plot_precision_recall_curve(y_true,y_pred,fig_name)
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1score = f1_score(y_true,y_pred,average="weighted")
        #print(f"accuracy {accuracy} precisions {precision} recall {recall} f1score {f1score}")
        return accuracy,precision,recall,f1score


In [ ]:
def multiple_train_time_metrics(list_ngrams, result_path, test_data, model_name, train_data,proj_number):
        final_result = {}
        log_file = f"{result_path}logs/trained_data_prec_rec_acc_{proj_number}_projects.txt"
        log_file_error = f"{result_path}logs/trained_data_prec_rec_acc_{proj_number}_projects_error.txt"
        real_model_name = f"{result_path}{model_name}"
        for each_gram in list_ngrams:
            for each_run in range(1,6):
                try:
                    # Log the time for training the model
                    train_start_time = time.time()  # Start time

                    train_mle(train_data, each_gram, real_model_name)
                    train_end_time = time.time()  # End time
                    train_duration = train_end_time - train_start_time  # Calculate duration

                    # Log the time for evaluating the model
                    eval_start_time = time.time()  # Start time

                    acc, precision, rec, f1_score = scratch_evaluate_model_nltk(test_data, f'{real_model_name}_{each_gram}.pkl')
                    eval_end_time = time.time()  # End time
                    eval_duration = eval_end_time - eval_start_time  # Calculate duration

                    # Store results
                    final_result[f'{each_gram}-gram_nltk'] = [acc, precision, rec, f1_score]

                    #Log training and evaluation results, including time
                    if not os.path.exists(log_file) or os.path.getsize(log_file) == 0:
                        with open(log_file,"a") as fl:
                            fl.write(f"ngram,run,accuracy,precision,recall,f1score,training_time,evaluation_time\n")

                    with open(log_file, "a") as precs:
                        precs.write(f"{each_gram},{each_run},{acc},{precision},{rec},{f1_score},{train_duration},{eval_duration}\n")

                except Exception as e:
                    # In case of an exception, log the error and mark the result as 0
                    final_result[f'{each_gram}-gram_nltk'] = [0, 0, 0, 0]
                    with open(log_file_error, "a") as precs:
                        precs.write(f"Error training or evaluating {each_gram}-gram model: {e}\n")

        return final_result

In [ ]:
multiple_train_time_metrics([2,3,4,5,6],"/content/results_500/","/content/scratch_test_data_20.txt","scratch_trained_model_nltk_500_projects","/content/scratch_train_data_500_projects.txt","500")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
